In [1]:
import os
import requests
import json
import openai
import pickle as pkl
import requests
import json
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from time import sleep
from tqdm.auto import tqdm
import numpy as np
import ast
from sentence_transformers import SentenceTransformer

from sklearn.manifold import TSNE
import plotly.express as px


/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''
Would need OPENAI Keys in a file called azure-configuration.json with params:
    key
    endpoint
    deployment_name
'''

#Checking if keys are working properly

with open('azure-configuration.json') as inputfile:
    azureconfig = json.load(inputfile)
openai.api_key = azureconfig['key'] 
openai.api_base = azureconfig['endpoint'] 
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name= azureconfig['deployment_name']

response = openai.Embedding.create(
    input="Your text string goes here",
    engine=deployment_name
)
embeddings = response['data'][0]['embedding']
print(len(embeddings))

1536


In [4]:
df_IAB=pd.read_csv('data/IAB_Evaluation.csv')   #Loading IAB Labels

In [5]:
iab_labels=df_IAB['IAB_Label'].to_list()

In [6]:
with open('data/ram_tag_list.txt', 'r') as f: #Loading RAM Labels
    check=f.readlines()

In [7]:
check=[x.split('\n')[0] for x in check] 

In [8]:
maf=check

In [9]:
call=iab_labels+maf

In [10]:
print(call[0:10])

['Hair Care', 'Shaving', 'Motorcycles', "Women's Formal Wear", "Women's Clothing", 'Retail Industry', 'Comedy Events', 'Household Supplies', 'Home Security', 'Polar Travel']


In [11]:
total_embeddings=[]

In [12]:
#Creating embeddings. Good to create it once and store it


def create_embeddings(item,total_embeddings):    
    deployment_name= azureconfig['deployment_name']

    response = openai.Embedding.create(
        input=item,
        engine=deployment_name
    )
    embeddings = response['data'][0]['embedding']
    total_embeddings.append(embeddings)

In [13]:
for items in tqdm(call):
    try:
        items=items.replace("&", 'and')
    except:
        pass

#only for Chat GPT version
    # if items in scores.keys():
    #     # print('here')
    #     key = list(scores[items].keys())[0]
    #     create_embeddings(key,total_embeddings)
    #     continue

    create_embeddings(items,total_embeddings)

100%|██████████| 4745/4745 [10:46<00:00,  7.34it/s] 


In [14]:
len(total_embeddings)

4745

In [15]:
iab_embeddings=total_embeddings[0:160]  #Change this if you want to test a new external keyword set

In [16]:
maf_embeddings=total_embeddings[160:]

In [17]:
nodes=maf

In [18]:
# with open("data/problematic.pkl", 'rb') as f:
#     problematic=pkl.load(f)

In [19]:
# with open("data/child_parent_dic.pkl", 'rb') as f:
#     child_parent_dic=pkl.load(f)

In [19]:
top_n_labels_new=[]
top_n_cosines_new=[]
top_n_nodes=[]
top_n_labels=[]
top_n_cosines=[]
tsne_embeddings=[]



for i in tqdm(range(len(maf_embeddings))):
    similarities=[]
    for j in range(len(iab_embeddings)):
        cos_sim=cosine_similarity(np.array(iab_embeddings[j]).reshape(1,-1),np.array(maf_embeddings[i]).reshape(1,-1))
        similarities.append([call[160+i],call[j],cos_sim])
    
    similarities.sort(key=lambda x: x[2], reverse=True)

    new_similarities=[x for x in similarities if x[2]>=0.765]   #LOW THRESHOLD TO SEE GENERAL CLUSTER VALUES

    if len(new_similarities)==0:
        new_similarities.append(['No_Pred','No_Pred',[[0]]])
    
    temp_iab_labels=[]
    temp_cosines=[]


    for items in new_similarities:
        tsne_embeddings.append(maf_embeddings[i])

        temp_iab_labels.append(items[1])
        temp_cosines.append(items[2][0])        
        top_n_labels_new.append(items[1])
        top_n_cosines_new.append(items[2][0])
        top_n_nodes.append(nodes[i])



    
    top_n_labels.append(temp_iab_labels)
    top_n_cosines.append(temp_cosines)



100%|██████████| 4585/4585 [06:33<00:00, 11.65it/s]


In [20]:
len(top_n_labels_new) #THIS IS THE NUMBER OF MAPPINGS. KEEPING IT HIGH HERE TO HAVE MORE SAMPLES FOR CLUSTERING

98890

In [21]:
dic={'item':top_n_nodes, 
'predictions': top_n_labels_new, 'cosine_predicted': top_n_cosines_new}

In [22]:
df_MAF_new=pd.DataFrame.from_dict(dic)

In [23]:
df_MAF_new.to_csv('results/RAM_INTERMEDIATE_0.785_TSNE_Mapping.csv')

In [24]:
df_tiers=pd.read_csv('data/MAFlabel_to_Adword_mappings_202210_combined_mappings.csv')

In [25]:
df_tiers=df_tiers['adword']

In [26]:
tier1=[]
all_tiers=df_tiers.to_list()
for tier in all_tiers:
    try:
        tier1.append(tier.split('|')[0])
    except:
        tier1.append(tier)

tier1=list(set(tier1))

In [27]:
len(tier1)

26

In [28]:
dic={'embeddings':tsne_embeddings, 'labels':top_n_labels_new, 'item': top_n_nodes}
df_labels_embeddings=pd.DataFrame.from_dict(dic)

In [29]:
df_labels_embeddings.head

<bound method NDFrame.head of                                               embeddings  \
0      [-0.014068884775042534, -0.0028959729243069887...   
1      [-0.014068884775042534, -0.0028959729243069887...   
2      [-0.014068884775042534, -0.0028959729243069887...   
3      [-0.014068884775042534, -0.0028959729243069887...   
4      [-0.014068884775042534, -0.0028959729243069887...   
...                                                  ...   
98885  [-0.016872776672244072, -0.017931459471583366,...   
98886  [-0.016872776672244072, -0.017931459471583366,...   
98887  [-0.016872776672244072, -0.017931459471583366,...   
98888  [-0.016872776672244072, -0.017931459471583366,...   
98889  [-0.016872776672244072, -0.017931459471583366,...   

                       labels             item  
0                Video Gaming  3D CG rendering  
1      Technology & Computing  3D CG rendering  
2                 Landscaping  3D CG rendering  
3                    Desktops  3D CG rendering  
4   

In [30]:
#Calculating tSNE clusters for each of the IAB labels.

results={}
for label in tqdm(iab_labels):
    mask = df_labels_embeddings['labels'].isin([label])
    df_tier1 = df_labels_embeddings[mask]
    if df_tier1.shape[0]//2<50:         #setting custom perplexity numbers
        perp=df_tier1.shape[0]//2       
    else:
        perp=50
    to_TSNE_list=df_tier1['embeddings'].to_list()
    to_TSNE_arr=np.array(to_TSNE_list)
    tsne=TSNE(n_components=2, random_state=42, perplexity=perp)
    X_embedded = tsne.fit_transform(to_TSNE_arr)
    results[label]=tsne.kl_divergence_


  0%|          | 0/160 [00:00<?, ?it/s]/Users/haskar140/anaconda3/envs/KG/lib/python3.11/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 160/160 [07:39<00:00,  2.87s/it]


In [31]:

problematic=[]
#new_problematic=[]
for k,v in results.items():
    if  v>1.5:   #Here v is an heuristic that I set after a bit of trial and error
        print(k,v)
        problematic.append(k)

Shaving 1.6874277591705322
Cooking 1.6342898607254028
Cruises 1.6424927711486816
Coupe 1.9804508686065674
Dating 1.9956855773925781
Wedding 1.8110712766647339
Outdoor Activities 1.652212381362915
Minivan 1.675191879272461
Travel 2.1230249404907227
Prom 1.9118818044662476
Science 2.0170462131500244
Convertible 2.0268921852111816
Camping 1.999372959136963
SUV 1.9113905429840088
Birds 1.941630482673645
Pets 1.5255075693130493
Sports 2.168769359588623
Shopping 2.1562047004699707
Houses 1.5803866386413574
Beauty 2.157377243041992
Graduation 2.042144775390625
Gardening 1.7481112480163574
Bath and Shower 1.5495142936706543
Wellness 1.7442004680633545
Television 1.5211454629898071
Station Wagon 1.6256574392318726
Desktops 1.7082254886627197
Movies 2.0206055641174316
Education 1.8935744762420654
Cats 1.6979376077651978


In [515]:
with open("data/problematic.pkl", 'wb') as f: #Save the problematic clusters to file
    pkl.dump(problematic,f)

In [42]:
problematic

['Shaving',
 'Cooking',
 'Cruises',
 'Coupe',
 'Dating',
 'Wedding',
 'Outdoor Activities',
 'Minivan',
 'Travel',
 'Prom',
 'Science',
 'Convertible',
 'Camping',
 'SUV',
 'Birds',
 'Pets',
 'Sports',
 'Shopping',
 'Houses',
 'Beauty',
 'Graduation',
 'Gardening',
 'Bath and Shower',
 'Wellness',
 'Television',
 'Station Wagon',
 'Desktops',
 'Movies',
 'Education',
 'Cats']

#CODE TO PLOT THE TIER1 CLUSTERS

'''


BELOW IS ONLY OPTIONAL



'''

In [32]:
mask = df_labels_embeddings['labels'].isin(tier1)
df_tier1 = df_labels_embeddings[mask]

In [33]:
df_tier1.head

<bound method NDFrame.head of                                               embeddings  \
0      [-0.014068884775042534, -0.0028959729243069887...   
1      [-0.014068884775042534, -0.0028959729243069887...   
4      [-0.014068884775042534, -0.0028959729243069887...   
5      [-0.014068884775042534, -0.0028959729243069887...   
6      [-0.014068884775042534, -0.0028959729243069887...   
...                                                  ...   
98867  [-0.016872776672244072, -0.017931459471583366,...   
98869  [-0.016872776672244072, -0.017931459471583366,...   
98879  [-0.016872776672244072, -0.017931459471583366,...   
98886  [-0.016872776672244072, -0.017931459471583366,...   
98887  [-0.016872776672244072, -0.017931459471583366,...   

                       labels             item  
0                Video Gaming  3D CG rendering  
1      Technology & Computing  3D CG rendering  
4                    Fine Art  3D CG rendering  
5                      Movies  3D CG rendering  
6   

In [34]:
df_tier1.shape

(23580, 3)

In [35]:
to_TSNE_list=df_tier1['embeddings'].to_list()

In [36]:
to_TSNE_arr=np.array(to_TSNE_list)

In [37]:
if df_tier1.shape[0]//2<50:
    perp=df_tier1.shape[0]//2
else:
    perp=50

In [38]:
tsne=TSNE(n_components=2, random_state=42, perplexity= perp) #df_tier1.shape[0]//2)
X_embedded = tsne.fit_transform(to_TSNE_arr)
tsne.kl_divergence_

1.0612928867340088

In [39]:
df_embeddings = pd.DataFrame(X_embedded)
df_embeddings = df_embeddings.rename(columns={0:'x',1:'y'})
df_embeddings = df_embeddings.assign(label=df_tier1.labels.values)
df_embeddings = df_embeddings.assign(items=df_tier1.item.values)

In [40]:
df_embeddings.head()

,x,y,label,items
0,-42.296238,115.41188,Video Gaming,3D CG rendering
1,-42.296238,115.41188,Technology & Computing,3D CG rendering
2,-42.296238,115.41188,Fine Art,3D CG rendering
3,-42.296238,115.41188,Movies,3D CG rendering
4,-42.296238,115.41188,Real Estate,3D CG rendering


In [41]:
fig = px.scatter(
    df_embeddings, x='x', y='y',
    color='label', labels={'color': 'label'},
    hover_data=['items'], title = 'Tier1 Adwords HTML')
fig.show()

In [513]:
fig.write_html("Tier1.html")
